<a href="https://colab.research.google.com/github/smduarte/spbd-2425/blob/main/docs/labs/lab9/SPBD_Labs_spark4_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Spark Streaming Exercises


In [ ]:
#@title Install Pyspark
!pip install --quiet pyspark

---
#Example
### Weblog Sender
The stream server is a small python TCP server, listening
on port 7777 (localhost).

The stream will consist of a set of text lines, obtained from the output log of a webserver.

In [ ]:
!wget -q -O - https://github.com/smduarte/spbd-2425/raw/main/scripts/logsender.tgz | tar xfz - 2> /dev/null

!nohup python logsender/server.py logsender/web.log 7777 > /dev/null 2> /dev/null &

## Spark UnStructured Streaming (Spark Core Streaming)

The python code below shows the basics needed to process data from socket source using PySpark.

Spark Streaming python documentation is found [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.streaming.html)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from pyspark.sql import *
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import tempfile

spark = SparkSession.builder.master('local[*]').appName('WebLogExample').getOrCreate()
sc = spark.sparkContext

try :
  ssc = StreamingContext.getActiveOrCreate(checkpointPath = tempfile.TemporaryDirectory().name, setupFunc = lambda : StreamingContext(sc, 1))
  lines = ssc.socketTextStream("localhost", 7777)

  lines.pprint()

  ssc.start()
  ssc.awaitTermination(10)
except Exception as err:
  ssc.stop()

---
# Exercises

Do the follwing exercises:

**Every 3 seconds**,
1. Dump the number of requests in the last 10 seconds;
2. Dump the number of requests in the last 10 seconds, only if they total more than 100;
3. Dump the number of requests in the last 10 seconds, if there is an IP address with more than 100 requests;
4. Dump the proportion of IPv4 vs IPv6 requests in the last 20 seconds.


# Structured Spark Streaming

The python code below shows the basics needed to process data from socket source using PySpark.

Spark Structured Streaming (Spark SQL Streaming) python documentation is found [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/index.html#)

In [ ]:
#@title Start the Structured Source

!wget -q -O - https://github.com/smduarte/spbd-2425/raw/main/scripts/json_logsender.tgz | tar xfz - 2> /dev/null

!nohup python json_logsender/server.py json_logsender/web.log 8888 > /dev/null 2> /dev/null &

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("StructuredWebLogExample") \
    .getOrCreate()


# Extract a sample JSON string to infer schema
sample_json = '{"timestamp": "2024-11-13T10:50:59.936+0000", "ip": "37.139.9.11", "code": 404, "cmd": "GET", "url": "/codemove/TTCENCUFMH3C", "time": 0.026}'
inferred_schema = schema_of_json(sample_json)


# Create DataFrame representing the stream of input
# lines from connection to logsender 7777
try:
  json_lines = spark.readStream.format("socket") \
      .option("host", "localhost") \
      .option("port", 8888) \
      .load()

  # Parse the JSON using the inferred schema
  json_lines = json_lines.withColumn("json_data", from_json(col("value"), inferred_schema)) \
    .select("json_data.*")  # Expand the JSON fields into columns


  query = json_lines \
    .writeStream \
    .outputMode("append") \
    .trigger(processingTime='1 seconds') \
    .foreachBatch(lambda df, epoch: df.show(10, False)) \
    .start()

  query.awaitTermination(60)
except Exception as err:
  print(err)
  query.stop()


---
# Exercises

Do the follwing exercises:

**Every 3 seconds**,
1. Dump the number of requests in the last 10 seconds;
2. Dump the number of requests in the last 10 seconds, only if they total more than 100;
3. Dump the number of requests in the last 10 seconds, if there is an IP address with more than 100 requests;
4. Dump the proportion of IPv4 vs IPv6 requests in the last 20 seconds.